In [ ]:
from Model import create_data,Model_TabNet,Model_NN,Model_prevailing,score
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
X_train,y_train,X_valid,y_valid,X_test,y_test=create_data('.\data.\processd_EPS1.csv','EPS1','Ptotal')

In [ ]:
TabNet=Model_TabNet(X_train, y_train,X_valid, y_valid,'Ptotal',16,16,500,100,1024,256)
MSE_Tabnet=score(TabNet,'Ptotal',X_test,y_test)


XGBoost,params1=Model_prevailing('XGBoost',X_train,y_train,'Ptotal')
MSE_XGBoost=score(XGBoost,'Ptotal',X_test,y_test)


RandomForest,params2=Model_prevailing('RandomForest',X_train,y_train,'Ptotal')
MSE_RandomForest=score(RandomForest,'Ptotal',X_test,y_test)


SVM,params3=Model_prevailing('SVM',X_train,y_train,'Ptotal')
MSE_SVM=score(SVM,'Ptotal',X_test,y_test)

NN=Model_NN(X_train,y_train,X_valid,y_valid,'Ptotal')
MSE_NN=score(NN,'Ptotal',X_test,y_test)

X_train1,y_train1,X_valid1,y_valid1,X_test1,y_test1=create_data('.\\data.\\unprocessed_EPS1.csv','EPS1','Ptotal')

TabNet1=Model_TabNet(X_train1, y_train1,X_valid1, y_valid1,'Ptotal',16,16,500,100,1024,256)
MSE_Tabnet1=score(TabNet1,'Ptotal',X_test1,y_test1)

print(f"The current value of MSE_Tabnet is{MSE_Tabnet}")
print(f"The current value of MSE_XGBoost is{MSE_XGBoost}")
print(f"The current value of MSE_RandomFores is{MSE_RandomForest}")
print(f"The current value of MSE_SVM is{MSE_SVM}")
print(f"The current value of MSE_NN is{MSE_NN}")
print(f"The current value of MSE_Tabnet1 is{MSE_Tabnet1}")

In [ ]:
#Global Feature Importance Evaluation

TabNet.feature_importance=TabNet.feature_importances_.reshape(1,3)
explain_matrix, masks = TabNet.explain(X_test)

fig, axs = plt.subplots(1, 1, figsize=(20, 20))
axs.imshow(TabNet.feature_importance)
axs.set_title(f"feature importance")

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20,20))
features = ["","P","", "Vref","","D1"]
for i in range(3):
    axs[i].imshow(masks[i][1000:1001])     #Select specific samples for analysis to evaluate the importance of each feature at each decision step.
    axs[i].set_title(f"mask {i}")
    axs[i].set_xticklabels(labels = features, rotation=45)

<div style="text-align: center;">
    <h1>    SHAP    </h1>
</div>

<div style="text-align: center;">
    <img src="image/SHAP.png" alt="SHAP" width="100"/>
</di)>

Using SHAP (SHapley Additive exPlanations) to perform interpretability analysis on a model built with TabNet allows for an in-depth understanding of the model's decision-making process at both global and individual levels. At the global level, analyzing all samples quantifies the overall contribution of each feature to the prediction outcomes, identifying which features have a positive or negative influence on the predictions. At the individual level, SHAP analysis can quantify the effect of each feature on the prediction for a specific sample, revealing how each feature contributes to the model's output for that instance. This interpretability analysis supports the transparency and reliability of the model, helping to identify potential biases and areas for optimizati

For more information, please refer to the [SHAP Official Documentation](https://shap.readthedocs.io/en/latest/index.html).on.

In [ ]:
import shap
column_names = ['P', 'Vref', 'D1']
temp_X_test = pd.DataFrame(X_test, columns=column_names)
explainer = shap.Explainer(TabNet.predict,X_test)
shap_values=explainer(X_test)
shap_values.feature_names=column_names

In [ ]:
shap.plots.waterfall(shap_values[1000])

In [ ]:
shap.initjs()
shap.plots.force(shap_values[0],matplotlib=True)

In [ ]:
shap.summary_plot(shap_values,X_test)

In [ ]:
shap.plots.scatter(shap_values[:, 0])

In [ ]:
shap.dependence_plot('P', shap_values.values,temp_X_test, interaction_index=None)

In [ ]:
shap.initjs()
shap.plots.force(shap_values.base_values[0],shap_values.values)